# Medical Insurance Cost Prediction

## Metadata

- age: age of primary beneficiary

- sex: insurance contractor gender, female, male

- bmi: Body mass index, providing an understanding of body, weights that are relatively high or low relative to height, objective index of body weight (kg / m ^ 2) using the ratio of height to weight, ideally 18.5 to 24.9

- children: Number of children covered by health insurance / Number of dependents

- smoker: Smoking

- region: the beneficiary's residential area in the US, northeast, southeast, southwest, northwest.

- charges: Individual medical costs billed by health insurance (Target)

## Pipeline

1. Import Necessary libraries
2. Data Collection
3. Exploratory Data Analysis
4. Preprocessing
5. Splitting
6. Training and prediction
7. Validation
8. Evaluation

### Import Necessary libraries

In [ ]:
# Data Manipulation
import pandas as pd
import numpy as np

#Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#Machine learning 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

### Data Collection

### EDA

### Preprocessing

### Splitting

### Training and Prediction

### Validation

### Evaluation